In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [4]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        if connection.is_connected():
            print("Successfully connected to the database")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection
# Gunakan fungsi ini untuk connect ke database
connection = create_connection("localhost", "root", "123454321", "spil")

Successfully connected to the database


In [5]:
cursor = connection.cursor(dictionary=True)


In [6]:
cursor.execute("SELECT * FROM data_all;")

In [7]:
table = cursor.fetchall()

In [8]:
df = pd.DataFrame(table)

In [9]:
pivot_table = df.pivot_table(index=['LOKASI', 'STATE'], 
                             columns=['TYPE', 'CONTAINER GRADE'],
                             aggfunc='size', 
                             fill_value=0)

# Reset the index for easier manipulation
pivot_table = pivot_table.reset_index()

In [10]:
def merge_pivot_for_display(pivot_df):
    # Identify columns that should be "merged"
    cols_to_merge = ['LOKASI', 'STATE']

    # Loop through the columns and replace duplicate values with an empty string
    for col in cols_to_merge:
        pivot_df[col] = pivot_df[col].where(pivot_df[col] != pivot_df[col].shift(), '')

    return pivot_df

# Apply this function to the pivot_table
pivot_table_for_display = merge_pivot_for_display(pivot_table)



In [11]:
pivot_table_for_display

TYPE                LOKASI STATE 20 CO          20 DC             ... 40 CO  \
CONTAINER GRADE                      -  B  C  X     -   A   B  C  ...     X   
0                AMB-AMBON   FAC     0  0  0  0     0  15  15  1  ...     0   
1                            FTL     0  0  0  0     0   0   3  0  ...     0   
2                            FXD     0  0  0  0     0  75  36  4  ...     0   
3                            MAS     0  0  0  0     0   0   0  1  ...     0   
4                            MIT     0  0  0  0     0  19   0  0  ...     0   
..                     ...   ...   ... .. .. ..   ...  ..  .. ..  ...   ...   
568                          MNA     0  0  0  1     0   0   2  3  ...     0   
569                          MOB     0  0  0  0     0  34  31  4  ...     0   
570                          MTA     0  0  0  0     0  31  31  9  ...     0   
571                          MTB     0  0  0  0     0   0   1  0  ...     0   
572                          STR     0  0  0  0     0  11   5  1  ...     0   

TYPE            40 DC 40 FT    40 HC       40 OT 40 RH 40 RM  
CONTAINER GRADE     X     C  X     A  B  C     X     X     C  
0                   0     0  0     2  0  0     0     0     0  
1                   0     0  0     0  0  0     0     0     0  
2                   0     0  2     3  2  5     0     0     0  
3                   0     0  0     0  0  0     0     0     0  
4                   0     0  0     0  0  0     0     0     0  
..                ...   ... ..   ... .. ..   ...   ...   ...  
568                 0     0  0     0  0  0     0     0     0  
569                 0     0  1     6  1  3     0     0     0  
570                 0     0  0     1  0  0     0     0     0  
571                 0     0  0     0  0  0     0     0     0  
572                 0     0  0     0  0  0     0     0     0  

[573 rows x 24 columns]

In [14]:
grouped = pivot_table_for_display.groupby(['STATE', 'TYPE', 'LOKASI', 'CONTAINER GRADE']).size().reset_index(name='count')

dict_grouped = grouped.to_dict(orient='records')
        
text = ''

for entry in dict_grouped:
    state = entry['STATE']
    type_ = entry['TYPE']  # Using 'type_' because 'type' is a reserved keyword in Python
    grade_ = entry['CONTAINER GRADE']
    loc = entry['LOKASI']
    count = entry['count']
    text += f'Jumlah kontainer dengan state {state} dan type {type_} dengan grade {grade_} yang berlokasi di {loc} adalah {count}, '

KeyError: 'TYPE'

In [25]:
tes  = df.groupby('STATE')['STATE'].agg('count')

In [22]:
text = ''

for key,value in tes.to_dict().items():
    text += f"jumlah kontainer dengan state {key} adalah {value}, "

In [23]:
text


'jumlah kontainer dengan state FAC adalah 632, jumlah kontainer dengan state FIT adalah 251, jumlah kontainer dengan state FOB adalah 3821, jumlah kontainer dengan state FTL adalah 3122, jumlah kontainer dengan state FXD adalah 2606, jumlah kontainer dengan state MAS adalah 762, jumlah kontainer dengan state MIT adalah 15, jumlah kontainer dengan state MNA adalah 868, jumlah kontainer dengan state MOB adalah 1515, jumlah kontainer dengan state MTA adalah 5408, jumlah kontainer dengan state MTB adalah 806, jumlah kontainer dengan state MTD adalah 3348, jumlah kontainer dengan state MTL adalah 792, jumlah kontainer dengan state MTS adalah 111, jumlah kontainer dengan state MXD adalah 12, jumlah kontainer dengan state STA adalah 1, jumlah kontainer dengan state STF adalah 4, jumlah kontainer dengan state STR adalah 156, '

In [30]:
grouped = df.groupby(['STATE', 'TYPE']).agg('count')

In [10]:
grouped = df.groupby(['STATE', 'TYPE', 'LOKASI']).size().reset_index(name='count')

In [11]:
dict_grouped = grouped.to_dict(orient='records')

In [12]:
dict_grouped

[{'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'AMB-AMBON', 'count': 16},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BAU-BAU', 'count': 2},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BIA-BIA', 'count': 4},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BIT-BITUNG', 'count': 5},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BKS-BKS', 'count': 5},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BLC-BLC', 'count': 6},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BLW-TPKDB', 'count': 43},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BMS-RTG', 'count': 19},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BOE-BOEPINANG', 'count': 2},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BPN-KKT', 'count': 33},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BRU-BERAU', 'count': 3},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'BTM-BTM', 'count': 10},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'JKT-217', 'count': 1},
 {'STATE': 'FAC', 'TYPE': '20 DC', 'LOKASI': 'JKT-LINI ADP', 'count': 34},
 {'STATE

In [13]:
text = ''

for entry in dict_grouped:
    state = entry['STATE']
    type_ = entry['TYPE']  # Using 'type_' because 'type' is a reserved keyword in Python
    loc = entry['LOKASI']
    count = entry['count']
    text += f'Jumlah kontainer dengan state {state} dan type {type_} dengan lokasi di {loc} adalah {count}, '

In [14]:
text

'Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di AMB-AMBON adalah 16, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BAU-BAU adalah 2, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BIA-BIA adalah 4, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BIT-BITUNG adalah 5, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BKS-BKS adalah 5, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BLC-BLC adalah 6, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BLW-TPKDB adalah 43, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BMS-RTG adalah 19, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BOE-BOEPINANG adalah 2, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BPN-KKT adalah 33, Jumlah kontainer dengan state FAC dan type 20 DC dengan lokasi di BRU-BERAU adalah 3, Jumlah kontainer dengan state FAC dan type 20 DC dengan 